In [19]:
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from statistics import mean, mode, median
from matplotlib import cm, colors
from distfit import distfit
from inflection import humanize, underscore
import json
from math import sqrt
import statistics


%matplotlib inline

In [20]:
def getData(metric:str, root_path:str):
    data = []
    for j in tqdm(range(len(os.listdir(root_path))), desc=metric):
        folder = os.listdir(root_path)[j]
        system_path = os.path.join(root_path, folder)
        metric_path = os.path.join(system_path, metric+'.csv')
    with open(metric_path, 'r') as csv_file:
            csv_reader = csv.reader(csv_file)
            for line in csv_reader:
                if(line[-1] != 'NaN' and line[-1] != 'nan'):
                    if(float(line[-1]) >= 0 ):
                        data.append(float(line[-1]))
    data = np.sort(data)
    
    return data

In [21]:
def statistic_metric(data, porcentagem):
    if (len(data) == 0) :
        return [0,0,0,0,0,0,(0,0)]
    metric_dict = []
    avg = np.average(data)
    median = np.median(data)
    mode = statistics.mode(data)
    std = np.std(data)
    x = porcentagem
    k = sqrt(-1/(x - 1))
    limite_inf = avg - k*std
    limite_sup = avg + k*std
    
    percentile70 = np.percentile(data, 70)
    percentile90 = np.percentile(data, 90)
    
    metric_dict = [
        avg,
        std,
        median,
        mode,
        percentile70,
        percentile90,
        (limite_inf, limite_sup)
    ]
    
    return metric_dict

In [22]:
def aparar_arr(arr, porcentagem):
    index_inf = round(len(arr) * porcentagem) - 1
    index_sup = round(len(arr) * (1-porcentagem)) - 1
    return arr[index_inf:index_sup]

In [23]:
def plotHist(data, metric, path, title, ext):
    plt.hist(data, bins=np.unique(data), log=True)
    plt.title(title)
    plt.xlabel("Metric value")
    plt.ylabel("Frequency")
    plt.savefig(path+'/hist_'+metric+ ext+'.png')
    plt.close()

### Tabela com todos os dados

In [25]:
project_path = 'C:/Users/h8men/OneDrive/Documentos/Cefet/TCC/metrics/'
folder = os.listdir(project_path)
metrics = os.listdir(project_path + f'{folder[0]}/')
arr = []
for i in range(len(metrics)):
    metric = metrics[i].replace('.csv', '')
    data = getData(metric,project_path)
    values = [metric]
    values.extend(statistic_metric(data,0.6))
    arr.append(values)


ABAGAIL:   1%|▏         | 1/80 [00:00<00:00, 332.99it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/h8men/OneDrive/Documentos/Cefet/TCC/metrics/Alink\\ABAGAIL.csv'

In [105]:
import pandas as pd
import os
project_path = 'C:/Users/h8men/OneDrive/Documentos/Cefet/TCC/metrics/'
dtframes = []
for file in os.listdir(project_path):
    if file.endswith("class.csv"):
        dfTemp = pd.read_csv(os.path.join(project_path, file))
        dtframes.append(dfTemp)


df = pd.concat(dtframes, axis=0, ignore_index=True)

C:\Users\h8men\AppData\Local\Temp\ipykernel_4052\4247444238.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dtframes, axis=0, ignore_index=True)


In [186]:
import functools
from math import sqrt

def calculate_metrics_on_df(dataframe_input):
    dfWithDataOnly = dataframe_input.drop(columns=['file', 'class', 'type', 'assignmen'])
    mean = dfWithDataOnly.mean()
    median = dfWithDataOnly.median()
    mode = dfWithDataOnly.mode().T
    mode['metric'] = mode.index
    std = dfWithDataOnly.std()
    percentile70 = dfWithDataOnly.quantile(0.7)
    percentile90 = dfWithDataOnly.quantile(0.9)

    x = 0.6
    k = sqrt(-1/(x - 1))
    limite_inf = mean - k*std
    limite_sup = mean + k*std

    meanDf = pd.DataFrame({'metric': mean.index, 'avg': mean.values})
    medianDf = pd.DataFrame({'metric': mean.index,'median': median.values})
    stdDf = pd.DataFrame({'metric': std.index, 'std': std.values})
    modeDf = mode.set_axis(['mode', 'metric'], axis=1)
    percentile70Df = pd.DataFrame({'metric': percentile70.index,'percentile70': percentile70.values})
    percentile90Df = pd.DataFrame({'metric': percentile90.index,'percentile90': percentile90.values})

    limite_infDf = pd.DataFrame({'metric': limite_inf.index, 'limite_inf': limite_inf.values})
    limite_supDf = pd.DataFrame({'metric': limite_sup.index, 'limite_sup': limite_sup.values})
    limites_merged = df_merged = pd.merge(limite_infDf, limite_supDf, on='metric')
    limites_merged['chebyshev'] = list(zip(df_merged['limite_inf'], df_merged['limite_sup']))
    chebyshev = limites_merged.drop(columns=['limite_inf', 'limite_sup'])

    metrics = [meanDf, stdDf, medianDf, modeDf, percentile70Df, percentile90Df, chebyshev]
    df_final = functools.reduce(lambda left, right: pd.merge(left,right,on='metric'), metrics)
    return df_final.sort_values(by='metric')
calculate_metrics_on_df(df)

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
19,abstractMethodsQty,0.055009,0.651543,0.0,0,0.0,0.0,"(-0.9751705801662092, 1.085189175575945)"
43,anonymousClassesQty,0.119011,0.990999,0.0,0.0,0.0,0.0,"(-1.4478963243784315, 1.6859175689972254)"
39,assignmentsQty,14.348358,79.476811,3.0,0.0,9.0,32.0,"(-111.31551447145542, 140.01223098061303)"
0,cbo,6.710141,7.876327,5.0,2,8.0,15.0,"(-5.7434250042778885, 19.16370673747197)"
1,cboModified,9.366113,12.427298,6.0,2,10.0,20.0,"(-10.28316967955523, 29.01539577935909)"
34,comparisonsQty,1.994027,8.792712,0.0,0,1.0,5.0,"(-11.908471909550709, 15.896524995624532)"
27,defaultFieldsQty,0.21842,1.387535,0.0,0,0.0,0.0,"(-1.9754651190925947, 2.412304408900782)"
17,defaultMethodsQty,0.784281,4.755282,0.0,0,0.0,2.0,"(-6.73448030063515, 8.303041321599565)"
5,dit,1.654189,2.480641,1.0,1,2.0,3.0,"(-2.268048254264796, 5.576426377911534)"
2,fanin,2.646496,8.511711,1.0,0,2.0,6.0,"(-10.811700092375684, 16.104692424953313)"


In [185]:
dfWithDataOnly = df.drop(columns=['file', 'class', 'type', 'assignmen'])
mode = dfWithDataOnly.mode().T
mode['metric'] = mode.index
mode.set_axis(['mode', 'metric'], axis=1)


,mode,metric
cbo,2,cbo
cboModified,2,cboModified
fanin,0,fanin
fanout,2,fanout
wmc,1,wmc
dit,1,dit
noc,0,noc
rfc,0,rfc
lcom,0,lcom
lcom*,0.0,lcom*


In [ ]:
df_data = pd.DataFrame(data=arr, columns=['metric', 'avg', 'std', 'median', 'mode', 'percentile70', 'percentile90', 'chebyshev'])
df_data

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
0,abstractMethodsQty,0.080690,0.835927,0.000000,0.0,0.000000,0.000000,"(-1.2410270205896254, 1.402406021636085)"
1,anonymousClassesQty,0.189588,1.825292,0.000000,0.0,0.000000,0.000000,"(-2.696452737186848, 3.0756281935594543)"
2,assignmentsQty,9.202378,32.831034,2.000000,0.0,5.000000,21.000000,"(-42.708044272759885, 61.11279984972449)"
3,cbo,10.385757,32.535123,5.000000,0.0,10.000000,22.000000,"(-41.05678970996809, 61.828304418537435)"
4,comparisonsQty,1.675236,9.543789,0.000000,0.0,0.000000,4.000000,"(-13.414819790482754, 16.76529235829499)"
5,defaultFieldsQty,0.211576,3.070956,0.000000,0.0,0.000000,0.000000,"(-4.644031348043464, 5.067183018226013)"
6,defaultMethodsQty,0.646554,3.844028,0.000000,0.0,0.000000,1.000000,"(-5.431387731433958, 6.724496048295244)"
7,dit,1.721110,1.712809,1.000000,1.0,2.000000,3.000000,"(-0.9870787026572885, 4.429299190008417)"
8,fanin,4.128229,28.923271,0.000000,0.0,2.000000,7.000000,"(-41.6034787783144, 49.85993609771708)"
9,fanout,6.257529,12.837907,3.000000,0.0,7.000000,15.000000,"(-14.040984150852532, 26.556041540019187)"


### Tabela com os dados > 1

In [ ]:
project_path = 'C:/Users/h8men/OneDrive/Documentos/Cefet/TCC/metrics/'
folder = os.listdir(project_path)
metrics = os.listdir(project_path + f'{folder[0]}/')
arr = []
for i in range(len(metrics)):
    metric = metrics[i].replace('.csv', '')
    data = getData(metric,project_path)
    new_data = []
    for value in data:
        if value > 0:
            new_data.append(value)
    values = [metric]
    values.extend(statistic_metric(new_data,0.6))
    arr.append(values)

ABAGAIL:   1%|▏         | 1/80 [00:00<00:00, 500.22it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/h8men/OneDrive/Documentos/Cefet/TCC/metrics/Alink/ABAGAIL.csv'

In [ ]:
values

['wmc',
 10.923737434311935,
 50.50972142227971,
 4.0,
 1.0,
 8.0,
 23.0,
 (-68.93914440318974, 90.7866192718136)]

In [ ]:
df_data1 = pd.DataFrame(data=arr, columns=['metric', 'avg', 'std', 'median', 'mode', 'percentile70', 'percentile90', 'chebyshev'])
df_data1

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
0,abstractMethodsQty,2.858761,4.100625,2.000000,1.0,3.0000,6.000000,"(-3.6248962959034188, 9.342418732272405)"
1,anonymousClassesQty,2.799890,6.472626,1.000000,1.0,2.0000,5.000000,"(-7.434230741729812, 13.034009964101944)"
2,assignmentsQty,14.583836,40.369864,5.000000,1.0,10.0000,32.000000,"(-49.246524509976055, 78.41419587988234)"
3,cbo,13.605339,36.645273,8.000000,3.0,13.0000,26.000000,"(-44.335925456360755, 71.54660294485531)"
4,comparisonsQty,6.285124,17.684837,2.000000,1.0,5.0000,13.000000,"(-21.67705756469917, 34.24730652864769)"
5,defaultFieldsQty,2.959800,11.126354,2.000000,1.0,3.0000,5.000000,"(-14.632510530038907, 20.552110855355508)"
6,defaultMethodsQty,3.215373,8.076226,1.000000,1.0,2.0000,6.000000,"(-9.554261777741349, 15.985007497750438)"
7,dit,1.721110,1.712809,1.000000,1.0,2.0000,3.000000,"(-0.9870787026572885, 4.429299190008417)"
8,fanin,9.256821,42.759247,3.000000,1.0,5.0000,15.000000,"(-58.35148469117851, 76.86512604916871)"
9,fanout,8.523363,14.324010,6.000000,2.0,9.0000,18.000000,"(-14.124884611846522, 31.171611046417635)"


### Tabela com dados aparados 5%

In [ ]:
project_path = '../../Projetos/'
folder = os.listdir(project_path)
metrics = os.listdir(project_path + f'{folder[0]}/')
arr = []
for i in range(len(metrics)):
    metric = metrics[i].replace('.csv', '')
    data = getData(metric,project_path)
    new_data = aparar_arr(data, 0.05)
    values = [metric]
    values.extend(statistic_metric(new_data,0.6))
    arr.append(values)

wmc: 100%|██████████| 44/44 [00:30<00:00,  1.45it/s]


In [ ]:
df_data5 = pd.DataFrame(data=arr, columns=['metric', 'avg', 'std', 'median', 'mode', 'percentile70', 'percentile90', 'chebyshev'])
df_data5

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
0,abstractMethodsQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
1,anonymousClassesQty,0.019674,0.138878,0.000000,0.0,0.000000,0.000000,"(-0.1999115712265472, 0.23926006935786268)"
2,assignmentsQty,4.883057,7.490827,2.000000,0.0,5.000000,15.000000,"(-6.960979844951196, 16.727094365322163)"
3,cbo,7.218092,7.249750,5.000000,0.0,9.000000,18.000000,"(-4.244769332183674, 18.680952601689512)"
4,comparisonsQty,0.604868,1.375443,0.000000,0.0,0.000000,2.000000,"(-1.5698980887061436, 2.779633704449234)"
5,defaultFieldsQty,0.023864,0.152625,0.000000,0.0,0.000000,0.000000,"(-0.21745711826223604, 0.2651847055828654)"
6,defaultMethodsQty,0.227680,0.562670,0.000000,0.0,0.000000,1.000000,"(-0.661979574345274, 1.1173387770398708)"
7,dit,1.504213,0.842605,1.000000,1.0,2.000000,3.000000,"(0.1719371396152478, 2.836488089933038)"
8,fanin,1.444226,2.475302,0.000000,0.0,1.000000,5.000000,"(-2.4695699591554288, 5.358021027304739)"
9,fanout,4.945301,5.107149,3.000000,0.0,7.000000,13.000000,"(-3.1298111717112027, 13.020412380588606)"


### Tabela com dados aparados 10%

In [ ]:
project_path = '../../Projetos/'
folder = os.listdir(project_path)
metrics = os.listdir(project_path + f'{folder[0]}/')
arr = []
for i in range(len(metrics)):
    metric = metrics[i].replace('.csv', '')
    data = getData(metric,project_path)
    new_data = aparar_arr(data, 0.1)
    values = [metric]
    values.extend(statistic_metric(new_data,0.6))
    arr.append(values)

wmc: 100%|██████████| 44/44 [00:22<00:00,  1.99it/s]


In [ ]:
df_data10 = pd.DataFrame(data=arr, columns=['metric', 'avg', 'std', 'median', 'mode', 'percentile70', 'percentile90', 'chebyshev'])
df_data10

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
0,abstractMethodsQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
1,anonymousClassesQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
2,assignmentsQty,3.707516,4.862352,2.000000,0.0,4.000000,11.000000,"(-3.980536617932502, 11.395569060185291)"
3,cbo,6.469338,5.632936,5.000000,0.0,9.000000,15.000000,"(-2.437116423327888, 15.375792779414054)"
4,comparisonsQty,0.352603,0.785381,0.000000,0.0,0.000000,1.000000,"(-0.8891932701438121, 1.5943996028715566)"
5,defaultFieldsQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
6,defaultMethodsQty,0.126348,0.332241,0.000000,0.0,0.000000,1.000000,"(-0.39897161941958137, 0.6516682194714889)"
7,dit,1.393418,0.660188,1.000000,1.0,1.000000,2.000000,"(0.34956985950404085, 2.437266372003875)"
8,fanin,1.039792,1.565162,0.000000,0.0,1.000000,3.000000,"(-1.4349462151456462, 3.514530954309934)"
9,fanout,4.422419,4.028875,3.000000,0.0,6.000000,11.000000,"(-1.9477920112073024, 10.792629799943352)"


In [ ]:
df_data.to_csv('./Statistic_Data_Complete.csv', sep=';')
df_data1.to_csv('./Statistic_Data_Without_0.csv', sep=';')
df_data5.to_csv('./Statistic_Data_Percentage5.csv', sep=';')
df_data10.to_csv('./Statistic_Data_Percentage10.csv', sep=';')



### Histogramas de NOC

In [ ]:
project_path = '../../Projetos/'
data = getData('noc',project_path)
project_path = './Hist/'
plotHist(data, 'noc', project_path,'Histograma NOC dataset completo', '')

noc: 100%|██████████| 44/44 [00:12<00:00,  3.40it/s]


In [ ]:
data_0 = []
for value in data:
    if value > 0:
        data_0.append(value)
plotHist(data_0, 'noc', project_path, 'Histograma NOC dataset > 0', '0')

In [ ]:
data_5 = aparar_arr(data, 0.05)
plotHist(data_5, 'noc', project_path,'Histograma NOC dataset 5% aparado', '5')

In [ ]:
data_10 = aparar_arr(data, 0.1)
plotHist(data_10, 'noc', project_path, 'Histograma NOC dataset 10% aparado', '10')

### Tabela com dados
Sem o valor 0, caso a métrica seja discreta e o percentil 70 tenha sido 0 também.

In [ ]:
metrics_without_zero = ['abstractMethodsQty', 'anonymousClassesQty', 'comparisonsQty','defaultFieldsQty', 'defaultMethodsQty','finalMethodsQty', 'innerClassesQty', 'lambdasQty', 'logStatementsQty', 'loopQty', 'mathOperationsQty', 'noc', 'parenthesizedExpsQty', 'privateMethodsQty', 'protectedFieldsQty', 'protectedMethodsQty', 'publicFieldsQty', 'staticFieldsQty', 'staticMethodsQty', 'synchronizedFieldsQty', 'synchronizedMethodsQty', 'tryCatchQty']

In [ ]:
def statistic_metric(data, porcentagem):
    if (len(data) == 0) :
        return [0,0,0,0,0,0,(0,0)]
    metric_dict = []
    avg = np.average(data)
    median = np.median(data)
    mode = statistics.mode(data)
    std = np.std(data)
    x = porcentagem
    k = sqrt(-1/(x - 1))
    limite_inf = avg - k*std
    limite_sup = avg + k*std
    
    percentile70 = np.percentile(data, 70)
    percentile90 = np.percentile(data, 90)
    
    metric_dict = [
        avg,
        std,
        median,
        mode,
        percentile70,
        percentile90
    ]
    
    return metric_dict

In [ ]:
project_path = '../../Projetos/'
folder = os.listdir(project_path)
metrics = os.listdir(project_path + f'{folder[0]}/')
arr = []
for i in range(len(metrics)):
    metric = metrics[i].replace('.csv', '')
    data = getData(metric,project_path)
    if (metric in metrics_without_zero):
        new_data = []
        for value in data:
            if value > 0:
                new_data.append(value)
        data = new_data
    values = [metric]
    values.extend(statistic_metric(data,0.6))
    arr.append(values)

wmc: 100%|██████████| 44/44 [00:36<00:00,  1.20it/s]


In [ ]:
df = pd.DataFrame(data= arr, columns=['metric', 'avg', 'std', 'median', 'mode', 'percentile70', 'percentile90', 'chebyshev'])
df

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
0,abstractMethodsQty,2.858761,4.100625,2.000000,1.0,3.000000,6.000000,None
1,anonymousClassesQty,2.799890,6.472626,1.000000,1.0,2.000000,5.000000,None
2,assignmentsQty,9.202378,32.831034,2.000000,0.0,5.000000,21.000000,None
3,cbo,10.385757,32.535123,5.000000,0.0,10.000000,22.000000,None
4,comparisonsQty,6.285124,17.684837,2.000000,1.0,5.000000,13.000000,None
5,defaultFieldsQty,2.959800,11.126354,2.000000,1.0,3.000000,5.000000,None
6,defaultMethodsQty,3.215373,8.076226,1.000000,1.0,2.000000,6.000000,None
7,dit,1.721110,1.712809,1.000000,1.0,2.000000,3.000000,None
8,fanin,4.128229,28.923271,0.000000,0.0,2.000000,7.000000,None
9,fanout,6.257529,12.837907,3.000000,0.0,7.000000,15.000000,None


In [ ]:
df.to_csv('./Statistic_Data_New_Approach.csv', sep=';', decimal=",")